In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
print(tf.__version__)
import string
import re
import nltk
from os import listdir
from collections import Counter
from numpy import array
import numpy as np
import pandas as pd
from nltk.corpus import stopwords

2.12.0


In [ ]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
!pip install --upgrade tensorflow-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# turn a doc into clean tokens
def clean_doc(doc):
# split into tokens by white space
    tokens = doc.split()
# prepare regex for char filtering
    re_punc = re.compile( ' [%s] ' % re.escape(string.punctuation))
# remove punctuation from each word
    tokens = [re_punc.sub( '' , w) for w in tokens]
# remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
# filter out stop words
    stop_words = set(stopwords.words( 'english' ))
    tokens = [w for w in tokens if not w in stop_words]
# filter out short tokens
    tokens = [word for word in tokens if len(word) > 1]
    return tokens


In [ ]:
# Load the dataset
train_df = pd.read_csv("/content/drive/MyDrive/NEEWWWWW/DATA_NEW/PC/1000/train_800.csv")
test_df = pd.read_csv("/content/drive/MyDrive/NEEWWWWW/DATA_NEW/PC/1000/test_200.csv")


In [ ]:
train_df[['text','label']].head()

,text,label
0,"Fast, no paper jams, prints clearly",1
1,"Fast PPM, Easy Set-up, Output is dry, ...",1
2,Hello Hewlett Packard,1
3,Great quality and speed,1
4,"Fast, sharp, reliable.",1


In [ ]:
train_df['label'].value_counts()

1    400
0    400
Name: label, dtype: int64

In [ ]:
train_pos_sentences = train_df['text'].loc[train_df.label==1]
train_neg_sentences = train_df['text'].loc[train_df.label==0]
train_pos_sentences = train_pos_sentences.reset_index(drop=True)
train_neg_sentences = train_neg_sentences.reset_index(drop=True)

test_pos_sentences = test_df['text'].loc[test_df.label==1]
test_neg_sentences = test_df['text'].loc[test_df.label==0]
test_pos_sentences = test_pos_sentences.reset_index(drop=True)
test_neg_sentences = test_neg_sentences.reset_index(drop=True)

In [ ]:
print(train_pos_sentences[0:5].values)

['  Fast, no paper jams, prints clearly '
 '        Fast PPM, Easy Set-up, Output is dry, great manual'
 '        Hello Hewlett Packard' '        Great quality and speed'
 '        Fast, sharp, reliable.']


In [ ]:
print(train_neg_sentences[0:5].values)

['  takes a few moments to change ink cartridges'
 '        Loud, cartridges have to be cleaned for real quality color'
 "        can't print, lousy support, bad communications, never again"
 '        Hard to find replacement cartridges'
 '        As always, expensive inks.']


In [ ]:
text = train_pos_sentences[0]
tokens = clean_doc(text)
print(tokens)

['paper', 'prints', 'clearly']


In [ ]:
# load doc and add to vocab
def add_sent_to_vocab(text,vocab):
# clean text
    tokens = clean_doc(text)
# update counts
    vocab.update(tokens)

def process_docs_to_vocab(sentList, vocab):
    for i in range(len(sentList)):
        text = sentList[i]
        add_sent_to_vocab(text,vocab)
       
# turn a doc into clean tokens
def clean_doc_wVocab(doc, vocab):
	# split into tokens by white space
	tokens = doc.split()
	# prepare regex for char filtering
	re_punc = re.compile('[%s]' % re.escape(string.punctuation))
	# remove punctuation from each word
	tokens = [re_punc.sub('', w) for w in tokens]
	# filter out tokens not in vocab
	tokens = [w for w in tokens if w in vocab]
	tokens = ' '.join(tokens)
	return tokens

# load all docs in a directory, into tokens
def process_docs_to_tokens(sentList, vocab):
	documents = list()
	# walk through all sentences
	for i in range(len(sentList)):
		doc = sentList[i]
		# clean doc
		tokens = clean_doc_wVocab(doc, vocab)
		# add to list
		documents.append(tokens)
	return documents

# integer encode and pad documents
def encode_docs(tokenizer, max_length, docs):
    # integer encode
    encoded = tokenizer.texts_to_sequences(docs)
    # pad sequences
    padded = tf.keras.preprocessing.sequence.pad_sequences(encoded, maxlen=max_length, padding='post')
    return padded

# fit a tokenizer
def create_tokenizer(lines):
	tokenizer = tf.keras.preprocessing.text.Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer

In [ ]:
# define vocab
vocab = Counter()
# add all docs to vocab
process_docs_to_vocab(train_df['text'], vocab)
process_docs_to_vocab(test_df['text'], vocab)
print(len(vocab))
# print the top words in the vocab
print(vocab.most_common(50))

1278
[('easy', 81), ('quality', 59), ('good', 47), ('print', 43), ('great', 42), ('ink', 34), ('use', 31), ('Great', 29), ('fast', 29), ('small', 29), ('paper', 27), ('Easy', 27), ('Good', 27), ('printing', 26), ('price', 25), ('Low', 24), ('Very', 22), ('Not', 22), ('printer', 21), ('No', 20), ('high', 19), ('little', 19), ('cartridges', 19), ('color', 18), ('child', 18), ('expensive', 18), ('hard', 18), ('low', 16), ('seat', 16), ('None', 16), ('slow', 16), ('prints', 15), ('The', 15), ('none', 15), ('well', 14), ('one', 14), ('Fast', 13), ('Excellent', 13), ('older', 13), ('inexpensive', 13), ('stroller', 13), ('Quality', 12), ('cheap', 12), ('It', 12), ('set', 12), ('cartridge', 12), ('Price', 11), ('large', 11), ('takes', 11), ('get', 11)]


In [ ]:
# load and clean a dataset
def load_clean_dataset(vocab):
	# load documents
	neg = process_docs_to_tokens(train_neg_sentences, vocab)
	pos = process_docs_to_tokens(train_pos_sentences, vocab)
	docs = neg + pos
	# prepare labels
	labels = array([0 for _ in range(len(neg))] + [1 for _ in range(len(pos))])
	return docs, labels

# load and clean a dataset
def load_clean_dataset_test(vocab):
	# load documents
	neg = process_docs_to_tokens(test_neg_sentences, vocab)
	pos = process_docs_to_tokens(test_pos_sentences, vocab)
	docs = neg + pos
	# prepare labels
	labels = array([0 for _ in range(len(neg))] + [1 for _ in range(len(pos))])
	return docs, labels

In [ ]:
# load training data
train_docs, ytrain = load_clean_dataset(vocab)
# create the tokenizer
tokenizer = create_tokenizer(train_docs)
# define vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary size: %d' % vocab_size)
# calculate the maximum sequence length
max_length = max([len(s.split()) for s in train_docs])
print('Maximum length: %d' % max_length)

Vocabulary size: 963
Maximum length: 16


In [ ]:
print(train_df['text'][0])

  Fast, no paper jams, prints clearly 


In [ ]:
train_docs[0]

'takes moments change ink cartridges'

In [ ]:
Xtrain = encode_docs(tokenizer, max_length, train_docs)

In [ ]:
print(Xtrain.shape)
print(Xtrain[0])

(800, 16)
[ 53 458 459  15  60   0   0   0   0   0   0   0   0   0   0   0]


In [ ]:
t = tokenizer
print(t.word_counts)
#print(t.document_count)
#print(t.word_index)
#print(t.word_docs)

OrderedDict([('takes', 12), ('moments', 1), ('change', 1), ('ink', 24), ('cartridges', 11), ('loud', 8), ('cleaned', 1), ('real', 2), ('quality', 86), ('color', 17), ('print', 40), ('lousy', 1), ('support', 11), ('bad', 6), ('never', 1), ('hard', 27), ('find', 4), ('replacement', 4), ('as', 4), ('always', 2), ('expensive', 24), ('inks', 2), ('running', 5), ('costs', 5), ('noisy', 5), ('large', 14), ('problems', 10), ('consumables', 2), ('somewhat', 8), ('no', 20), ('cons', 1), ('come', 1), ('mind', 1), ('still', 4), ('around', 3), ('price', 47), ('constant', 1), ('maintenance', 4), ('pricey', 8), ('cartridge', 6), ('monitor', 1), ('system', 5), ('leaves', 1), ('something', 1), ('not', 21), ('one', 18), ('slow', 13), ('photos', 10), ('medium', 1), ('regular', 2), ('paper', 29), ('hog', 1), ('epson', 1), ('lines', 1), ('output', 15), ('found', 4), ('minor', 4), ('nozzle', 1), ('clogging', 1), ('issue', 2), ('situations', 1), ('blurred', 1), ('picture', 7), ('none', 29), ('use', 54), ('le

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 63.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 53.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 22.4 MB/s eta 0:00:00


In [ ]:
# import tensorflow as tf
# from transformers import TFAutoModel, AutoTokenizer

# def define_model(length, vocab_size):
#     # channel 1
#     inputs1 = tf.keras.layers.Input(shape=(length,))
#     embedding1 = tf.keras.layers.Embedding(vocab_size, 100)(inputs1)
#     conv1 = tf.keras.layers.Conv1D(filters=32, kernel_size=4, activation='relu')(embedding1)
#     drop1 = tf.keras.layers.Dropout(0.5)(conv1)
#     pool1 = tf.keras.layers.MaxPooling1D(pool_size=2)(drop1)
#     flat1 = tf.keras.layers.Flatten()(pool1)
#     # channel 2
#     inputs2 = tf.keras.layers.Input(shape=(length,))
#     tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
#     bert_model = TFAutoModel.from_pretrained('bert-base-uncased')
#     input_ids = tf.keras.layers.Input(shape=(length,), dtype=tf.int32, name="input_ids")
#     bert_embedding = bert_model(input_ids)[0]
#     pooled_output = bert_embedding[:, 0, :]
#     dense2 = tf.keras.layers.Dense(100, activation='relu')(pooled_output)
#     # channel 3
#     inputs3 = tf.keras.layers.Input(shape=(length,))
#     embedding3 = tf.keras.layers.Embedding(vocab_size, 100)(inputs3)
#     lstm3 = tf.keras.layers.LSTM(64)(embedding3)
#     dense3 = tf.keras.layers.Dense(100, activation='relu')(lstm3)
#     # merge
#     merged = tf.keras.layers.concatenate([flat1, dense2, dense3])
#     # interpretation
#     dense4 = tf.keras.layers.Dense(10, activation='relu')(merged)
#     outputs = tf.keras.layers.Dense(1, activation='sigmoid')(dense4)
#     model = tf.keras.models.Model(inputs=[inputs1, input_ids, inputs3], outputs=outputs)
#     # compile
#     model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#     # summarize
#     print(model.summary())
#     return model

In [ ]:
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer
from gensim.models import KeyedVectors

def define_model(length, vocab_size, word2vec_model_path):
    # channel 1
    inputs1 = tf.keras.layers.Input(shape=(length,), dtype=tf.int32)
    tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
    input_ids = tf.keras.layers.Input(shape=(length,), dtype=tf.int32)
    bert_model = TFAutoModel.from_pretrained('bert-base-uncased')
    bert_output = bert_model(input_ids)[0] # use only the first tensor returned by the BERT model
    pooled_output = bert_output[:, 0, :]
    dense1 = tf.keras.layers.Dense(100, activation='relu')(pooled_output)

    # channel 2
    inputs2 = tf.keras.layers.Input(shape=(length,))
    word2vec_model = KeyedVectors.load_word2vec_format(word2vec_model_path, binary=True)
    vocab_size = min(vocab_size, len(word2vec_model.key_to_index)) # adjust vocab_size based on actual vocabulary size
    embedding_matrix = np.zeros((vocab_size, word2vec_model.vector_size))
    for word, index in word2vec_model.key_to_index.items():
        if index >= vocab_size:
            continue
        embedding_matrix[index] = word2vec_model[word]
    embedding2 = tf.keras.layers.Embedding(vocab_size, word2vec_model.vector_size, weights=[embedding_matrix], trainable=False)(inputs2)
    conv2 = tf.keras.layers.Conv1D(filters=32, kernel_size=4, activation='relu')(embedding2)
    drop2 = tf.keras.layers.Dropout(0.5)(conv2)
    pool2 = tf.keras.layers.MaxPooling1D(pool_size=2)(drop2)
    flat2 = tf.keras.layers.Flatten()(pool2)

    # channel 3
    inputs3 = tf.keras.layers.Input(shape=(length,))
    embedding3 = tf.keras.layers.Embedding(vocab_size, 100)(inputs3)
    lstm3 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64))(embedding3)
    dense3 = tf.keras.layers.Dense(100, activation='relu')(lstm3)

    # merge
    merged = tf.keras.layers.concatenate([dense1, flat2, dense3])

    # interpretation
    dense4 = tf.keras.layers.Dense(10, activation='relu')(merged)
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(dense4)

    model = tf.keras.models.Model(inputs=[input_ids, inputs2, inputs3], outputs=outputs)

    # compile
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    # summarize
    print(model.summary())

    return model

WORD2VEC_MODEL = '/content/drive/MyDrive/NEEWWWWW/GoogleNews-vectors-negative300.bin'
model = define_model(length=100, vocab_size=10000, word2vec_model_path=WORD2VEC_MODEL)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 100)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 100, 300)     3000000     ['input_3[0][0]']                
                                                                                                  
 input_2 (InputLayer)           [(None, 100)]        0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 97, 32)       38432       ['embedding[0][0]']              
                                                                                              

In [ ]:
word2vec_model_path = '/content/drive/MyDrive/NEEWWWWW/GoogleNews-vectors-negative300.bin'
print(vocab_size)
print(max_length)
# define model
model = define_model(max_length, vocab_size, word2vec_model_path)

963
16


Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 16)]         0           []                               
                                                                                                  
 embedding_2 (Embedding)        (None, 16, 300)      288900      ['input_7[0][0]']                
                                                                                                  
 input_6 (InputLayer)           [(None, 16)]         0           []                               
                                                                                                  
 conv1d_1 (Conv1D)              (None, 13, 32)       38432       ['embedding_2[0][0]']            
                                                                                            

In [ ]:
# fit model
model.fit([Xtrain,Xtrain,Xtrain], ytrain, [Xtest,Xtest,Xtest], ytest, epochs=10)

Epoch 1/10


25/25 [==============================] - 227s 8s/step - loss: 0.7695 - accuracy: 0.5150
Epoch 2/10
25/25 [==============================] - 167s 7s/step - loss: 0.6696 - accuracy: 0.6162
Epoch 3/10
25/25 [==============================] - 166s 7s/step - loss: 0.4643 - accuracy: 0.8550
Epoch 4/10
25/25 [==============================] - 177s 7s/step - loss: 0.1864 - accuracy: 0.9400
Epoch 5/10
25/25 [==============================] - 166s 7s/step - loss: 0.0990 - accuracy: 0.9700
Epoch 6/10
25/25 [==============================] - 169s 7s/step - loss: 0.0877 - accuracy: 0.9750
Epoch 7/10
25/25 [==============================] - 166s 7s/step - loss: 0.0572 - accuracy: 0.9800
Epoch 8/10
25/25 [==============================] - 169s 7s/step - loss: 0.0511 - accuracy: 0.9825
Epoch 9/10
25/25 [==============================] - 167s 7s/step - loss: 0.0342 - accuracy: 0.9887
Epoch 10/10
25/25 [==============================] - 168s 7s/step - loss: 0.0377 - accuracy: 0.9875


In [ ]:
test_docs, ytest = load_clean_dataset_test(vocab)

In [ ]:
Xtest = encode_docs(tokenizer, max_length, test_docs)

In [ ]:
# evaluate model on training dataset
_, acc = model.evaluate([Xtrain,Xtrain,Xtrain], ytrain, verbose=0)
print(' Train Accuracy: %f' % (acc*100))

 Train Accuracy: 99.000001


In [ ]:
# evaluate model on test dataset
_, acc = model.evaluate([Xtest,Xtest,Xtest], ytest, verbose=0)
print( ' Test Accuracy: %f ' % (acc*100))
     

 Test Accuracy: 87.500000 


In [ ]:
model.save('/content/gdrive/My Drive/sent_model_multi_input.h5')